In [3]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

print(len(train_data))
print(len(test_data))
print(test_data[10])

8982
2246
[1, 361, 372, 8, 77, 62, 325, 4105, 336, 5, 1605, 37, 412, 453, 1187, 229, 334, 13, 4, 867, 76, 4, 76, 1726, 6, 264, 2337, 18, 82, 95, 97, 2220, 4, 1004, 649, 18, 82, 554, 136, 4, 143, 334, 290, 126, 5, 4, 2820, 777, 2, 1386, 13, 954, 7, 4, 314, 912, 224, 4, 2370, 1128, 54, 429, 2, 18, 82, 5, 496, 1187, 229, 57, 85, 385, 593, 6, 4, 867, 76, 17, 12]


In [4]:
# decode
word_index = reuters.get_word_index()
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])
decoded = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])

print(decoded)
print(train_labels[0])

? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3
3


In [5]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [6]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

print(one_hot_train_labels[0])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
# or using Keras
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

print(one_hot_train_labels[0])


[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
#  set aside validation set
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [14]:
# train
history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

Train on 7982 samples, validate on 1000 samples
Epoch 1/20
7982/7982 [==============================] - 4s 526us/step - loss: 2.7090 - accuracy: 0.4678 - val_loss: 1.7525 - val_accuracy: 0.6500
Epoch 2/20
7982/7982 [==============================] - 2s 283us/step - loss: 1.4284 - accuracy: 0.7068 - val_loss: 1.3049 - val_accuracy: 0.7150
Epoch 3/20
7982/7982 [==============================] - 2s 191us/step - loss: 1.0528 - accuracy: 0.7762 - val_loss: 1.1280 - val_accuracy: 0.7560
Epoch 4/20
7982/7982 [==============================] - 2s 254us/step - loss: 0.8372 - accuracy: 0.8201 - val_loss: 1.0356 - val_accuracy: 0.7710
Epoch 5/20
7982/7982 [==============================] - 2s 287us/step - loss: 0.6684 - accuracy: 0.8583 - val_loss: 1.0284 - val_accuracy: 0.7760
Epoch 6/20
7982/7982 [==============================] - 2s 303us/step - loss: 0.5382 - accuracy: 0.8875 - val_loss: 0.9249 - val_accuracy: 0.8050
Epoch 7/20
7982/7982 [==============================] - 2s 307us/step - loss

In [15]:
# plot training and validation results
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>